In [1]:
import numpy as np
import pandas as pd
import random

complete = {"a":"t", "t": "a", "g":"c","c":"g"}
getComplete = lambda info: ''.join(map(lambda x: complete[x],info))

class DNA_strand:
    def __init__(self, genInfo, completeInfo = ""):
        self.genInfo = genInfo
        if completeInfo == "":
            self.completeInfo = getComplete(genInfo)
        else:
            self.completeInfo = completeInfo
        
    def isSame(self, DNA_strand):
        return DNA_strand.genInfo ==  getComplete(self.genInfo)
    
    def mix(self, seq):
        if np.random.choice([1, 1, 1, 1, 1, 1, 1, 1, 1, 0]) == 1:
            self.genInfo += seq.genInfo
            self.completeInfo += seq.completeInfo
    
class DNA_lab:
    def __init__(self, strandArray = []):
        self.strandArray = strandArray
    
    def extract(self, specificStrand = [], Partially = False):
        if Partially:
            res = []
            for j in self.strandArray:
                for i in specificStrand:
                    if i in j.genInfo:
                        res.append(j)
                        break
        else:
            res = list(filter(lambda strand: strand.genInfo in specificStrand,self.strandArray))
        self.strandArray = res
        return res
    
    def length_sort(self):
        return sorted(self.strandArray, key=lambda x: len(x.genInfo), reverse=True)
    
    def Amplify(self,sequence, CycleNum = 1):
        prev = [sequence]
        for i in range(CycleNum):
            new_prev = []
            for seq in prev:
                if np.random.choice([1, 1, 1, 1, 1, 1, 1, 1, 1, 0]) == 1:
                    new_prev.append(DNA_strand(seq.genInfo))
                    new_prev.append(DNA_strand(getComplete(seq.completeInfo)))
            prev = new_prev
            
        self.strandArray  += prev
        return prev
    
    def sequence(self,index):
        return strandArray[index].genInfo
    
    def cleave(seq, subSeq, i1, i2):
        self.strandArray.remove(seq)
        part1 , part2 = None, seq
        while True:
            try:
                index = part2.genInfo.index(subSeq)
                part1, part2 = DNA_strand(part2.genInfo[:index+i1], part2.completeInfo[:index+i2]) , DNA_strand(part2.genInfo[index+i1:], part2.completeInfo[index+i2:])
                self.strandArray.append(part1)
            except:
                self.strandArray.append(part2)

In [2]:
def getUnique(sat):
    unique = []
    for i in sat:
        for j in i:
            if j[-1] not in unique:
                unique.append(j[-1])
    return unique

def getVertexes(array):
    new = []
    for i in range(1,len(array)+2):
        new.append('a'+str(i))
    array.sort()
    return array + new + list(map(lambda x: '-' + x, array))

def getEdges(vertex):
    counter = 1
    edges = []
    for v in vertex:
        if 'a' not in v and '-' not in v:
            if 'a'+str(counter) in vertex:
                edges.append(('a'+str(counter),v))
                edges.append(('a'+str(counter),'-'+v))
                edges.append((v,'a'+str(counter+1)))
                edges.append(('-'+v,'a'+str(counter+1)))
                counter += 1
    return edges

def BuildGraph(sat):
    v = getVertexes(getUnique(sat))
    e = getEdges(v)
    return {'V': v, 'E': e}

def truthTable(sat):
    length =  len(getUnique(sat))
    limit = 2**(length)

    result = np.zeros((limit, length))

    for i in range(length-1,-1,-1):
        IsOne = True
        for j in range(limit):
            if j % 2**i == 0 and j!= 0:
                IsOne = not IsOne
        
            if IsOne:
                result[j,length - i - 1] = 1
            else:
                result[j,length - i -1] = 0

    return pd.DataFrame(result, columns = getUnique(sat))

sat = [
    ["w","x","y"],
    ["w","y","-z"],
    ["w","-y","z"],
    ["-w","-x","-z"]
]

Graph = BuildGraph(sat)
TruthTable = truthTable(sat)

In [3]:
def GetPaths(df):
    paths = {}
    for index, row in df.iterrows():
        path = []
        for i, v in enumerate(row):
            if v == 1:
                path.append(("a"+str(i+1),df.columns[i]))
                path.append((df.columns[i],"a"+str(i+2)))
            else:
                path.append(("a"+str(i+1),"-"+df.columns[i]))
                path.append(("-"+df.columns[i],"a"+str(i+2)))
        paths[index] = path
    return paths
        
def Vertex2Seq(G):
    res = {}
    for v in G['V']:
        res[v] = ''.join(random.choices(['a','t','c','g'], weights = [1,1, 1, 1], k = 20))
    return res

def Path2Seq(path,V2S):
    result = V2S[path[0][0]]
    for i in path:
        result += V2S[i[1]]
    return result

def Seq2Path(seq,V2S):
    res = []
    S2V = {list(V2S.values())[i]: list(V2S.keys())[i] for i in range(len(V2S.keys()))}
    for i in range(0,len(seq.genInfo),20):
        res.append(S2V[seq.genInfo[i:i+20]])
    return res
        
paths = GetPaths(TruthTable)
V2S = Vertex2Seq(Graph)

strandArray = []

for path in paths.values():
    strandArray.append(DNA_strand(Path2Seq(path,V2S)))
    
DNALab = DNA_lab(strandArray)

for i,C in enumerate(sat):
    seq = list(map(lambda x: V2S[x], C))
    DNALab.extract(seq, True)
    print("After C{} (len = {}):\n".format(i+1,len(DNALab.strandArray)))
    for strand in DNALab.strandArray:
        print(Seq2Path(strand,V2S)[1:-1:2])
    print("")

After C1 (len = 14):

['w', 'x', 'y', 'z']
['w', 'x', 'y', '-z']
['w', 'x', '-y', 'z']
['w', 'x', '-y', '-z']
['w', '-x', 'y', 'z']
['w', '-x', 'y', '-z']
['w', '-x', '-y', 'z']
['w', '-x', '-y', '-z']
['-w', 'x', 'y', 'z']
['-w', 'x', 'y', '-z']
['-w', 'x', '-y', 'z']
['-w', 'x', '-y', '-z']
['-w', '-x', 'y', 'z']
['-w', '-x', 'y', '-z']

After C2 (len = 13):

['w', 'x', 'y', 'z']
['w', 'x', 'y', '-z']
['w', 'x', '-y', 'z']
['w', 'x', '-y', '-z']
['w', '-x', 'y', 'z']
['w', '-x', 'y', '-z']
['w', '-x', '-y', 'z']
['w', '-x', '-y', '-z']
['-w', 'x', 'y', 'z']
['-w', 'x', 'y', '-z']
['-w', 'x', '-y', '-z']
['-w', '-x', 'y', 'z']
['-w', '-x', 'y', '-z']

After C3 (len = 11):

['w', 'x', 'y', 'z']
['w', 'x', 'y', '-z']
['w', 'x', '-y', 'z']
['w', 'x', '-y', '-z']
['w', '-x', 'y', 'z']
['w', '-x', 'y', '-z']
['w', '-x', '-y', 'z']
['w', '-x', '-y', '-z']
['-w', 'x', 'y', 'z']
['-w', 'x', '-y', '-z']
['-w', '-x', 'y', 'z']

After C4 (len = 9):

['w', 'x', 'y', '-z']
['w', 'x', '-y', '-z']
[